In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline
import seaborn as sns
sns.set()
sns.set_style("darkgrid")
import warnings
warnings.filterwarnings("ignore")
import math
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold

In [3]:
covid = pd.read_csv('Data-Covid002.csv', engine='python',encoding='latin1')
covid.head()

,county,state,fips,cases,deaths,population,casespc,deathspc,cty,county_name,...,taxrate,tax_st_diff_top20,summer_tmmx,summer_rmax,winter_tmmx,winter_rmax,pm25,bmdeaths,bmcruderate,pm25_mia
0,Autauga,Alabama,1001,4434,152,918492,482.74780,16.548864,1001,Autauga,...,0.011183,0.0,306.02344,96.055420,288.08508,85.651848,11.712587,7893,859.29999,0
1,Baldwin,Alabama,1003,10465,278,3102984,337.25601,8.959118,1003,Baldwin,...,0.011756,0.0,305.51663,97.971542,290.20886,89.730972,10.077723,30292,976.20001,0
2,Barbour,Alabama,1005,3157,33,499262,632.33331,6.609756,1005,Barbour,...,0.012059,0.0,306.06226,97.371674,289.24210,88.633575,10.981967,5197,1040.90000,0
3,Bibb,Alabama,1007,2291,24,397470,576.39569,6.038192,1007,Bibb,...,0.008007,0.0,305.98218,96.293076,287.36282,86.485870,11.998714,4089,1028.80000,0
4,Blount,Alabama,1009,2082,15,997531,208.71532,1.503713,1009,Blount,...,0.007012,0.0,305.17886,94.630951,285.56567,85.449142,11.793022,9912,993.70001,0


In [4]:
covid.columns

Index(['county', 'state', 'fips', 'cases', 'deaths', 'population', 'casespc',
       'deathspc', 'cty', 'county_name', 'cty_pop2000', 'cz', 'cz_name',
       'cz_pop2000', 'statename', 'state_id', 'stateabbrv', 'csa', 'csa_name',
       'cbsa', 'cbsa_name', 'intersects_msa', 'cur_smoke_q1', 'cur_smoke_q2',
       'cur_smoke_q3', 'cur_smoke_q4', 'bmi_obese_q1', 'bmi_obese_q2',
       'bmi_obese_q3', 'bmi_obese_q4', 'exercise_any_q1', 'exercise_any_q2',
       'exercise_any_q3', 'exercise_any_q4', 'brfss_mia', 'puninsured2010',
       'reimb_penroll_adj10', 'mort_30day_hosp_z', 'adjmortmeas_amiall30day',
       'adjmortmeas_chfall30day', 'adjmortmeas_pnall30day', 'med_prev_qual_z',
       'primcarevis_10', 'diab_hemotest_10', 'diab_eyeexam_10',
       'diab_lipids_10', 'mammogram_10', 'amb_disch_per1000_10',
       'cs00_seg_inc', 'cs00_seg_inc_pov25', 'cs00_seg_inc_aff75',
       'cs_race_theil_2000', 'gini99', 'poor_share', 'inc_share_1perc',
       'frac_middleclass', 'scap_ski90pcm',

In [5]:
covid.drop(['fips', 'casespc', 'cases', 'deaths', 'population', 'cty', 'county_name', 'cty_pop2000', 'cz', 'cz_name',
       'cz_pop2000', 'statename', 'state_id', 'stateabbrv', 'csa', 'csa_name',
       'cbsa', 'cbsa_name',], axis = 1, inplace=True)
covid.head()

,county,state,deathspc,intersects_msa,cur_smoke_q1,cur_smoke_q2,cur_smoke_q3,cur_smoke_q4,bmi_obese_q1,bmi_obese_q2,...,taxrate,tax_st_diff_top20,summer_tmmx,summer_rmax,winter_tmmx,winter_rmax,pm25,bmdeaths,bmcruderate,pm25_mia
0,Autauga,Alabama,16.548864,1,0.333333,0.238095,0.208333,0.133333,0.375000,0.238095,...,0.011183,0.0,306.02344,96.055420,288.08508,85.651848,11.712587,7893,859.29999,0
1,Baldwin,Alabama,8.959118,1,0.268097,0.233503,0.167464,0.176991,0.298050,0.262467,...,0.011756,0.0,305.51663,97.971542,290.20886,89.730972,10.077723,30292,976.20001,0
2,Barbour,Alabama,6.609756,0,0.228571,0.250000,0.181818,0.111111,0.294118,0.571429,...,0.012059,0.0,306.06226,97.371674,289.24210,88.633575,10.981967,5197,1040.90000,0
3,Bibb,Alabama,6.038192,1,0.244444,0.280000,0.181818,0.150000,0.466667,0.375000,...,0.008007,0.0,305.98218,96.293076,287.36282,86.485870,11.998714,4089,1028.80000,0
4,Blount,Alabama,1.503713,1,0.304348,0.260870,0.352941,0.166667,0.347826,0.318182,...,0.007012,0.0,305.17886,94.630951,285.56567,85.449142,11.793022,9912,993.70001,0


In [6]:
covid.describe()

,deathspc,intersects_msa,cur_smoke_q1,cur_smoke_q2,cur_smoke_q3,cur_smoke_q4,bmi_obese_q1,bmi_obese_q2,bmi_obese_q3,bmi_obese_q4,...,taxrate,tax_st_diff_top20,summer_tmmx,summer_rmax,winter_tmmx,winter_rmax,pm25,bmdeaths,bmcruderate,pm25_mia
count,3107.000000,3107.000000,3107.000000,3107.000000,3107.000000,3107.000000,3107.000000,3107.000000,3107.000000,3107.000000,...,3107.000000,3106.000000,3107.000000,3107.000000,3107.000000,3107.000000,3107.000000,3.107000e+03,3107.00000,3107.000000
mean,23.790131,0.596717,0.212659,0.171048,0.134467,0.098316,0.239166,0.214580,0.209621,0.186739,...,0.023089,0.775634,303.126997,88.970517,280.404875,87.469432,8.371871,1.437848e+04,1029.15597,0.003540
std,67.852145,0.490636,0.149348,0.128130,0.132181,0.110110,0.165928,0.153237,0.175849,0.167227,...,0.013848,1.470989,3.173950,9.689271,6.597855,4.811207,2.565927,3.918052e+04,248.38181,0.059405
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,290.455540,31.643282,264.693820,58.159798,0.000000,1.100000e+01,189.30000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080128,0.000000,0.000000,0.000000,...,0.014993,0.000000,300.848035,88.052494,275.113020,85.093342,6.309710,2.235500e+03,864.29999,0.000000
50%,3.802303,1.000000,0.250000,0.198718,0.142857,0.096535,0.272076,0.241590,0.223124,0.194118,...,0.020339,0.000000,303.290440,91.320313,280.154690,88.028793,8.784647,4.792000e+03,1036.30000,0.000000
75%,21.461759,1.000000,0.310931,0.250000,0.200000,0.148719,0.335532,0.304348,0.297220,0.266667,...,0.027164,1.000000,305.817430,94.812389,285.543750,90.747704,10.483764,1.122500e+04,1194.10000,0.000000
max,2279.610600,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.209907,7.220000,313.872680,99.778748,298.340360,97.672874,15.786018,1.072024e+06,1978.60000,1.000000


In [8]:
covid.isnull().sum()!=0

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [ ]:
covid.dropna(inplace=True)
